# Half Marathon Time Predictor - Training Pipeline

Ten notebook zawiera kompletny pipeline do trenowania modelu przewidującego czas ukończenia półmaratonu.

## Kroki:
1. Wczytanie danych z Digital Ocean Spaces
2. Czyszczenie i przygotowanie danych
3. Feature engineering i selekcja
4. Trenowanie modelu
5. Walidacja i metryki
6. Zapisanie modelu lokalnie i w Digital Ocean Spaces

In [ ]:
# Import bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import joblib
import sys
from pathlib import Path

# Dodaj główny katalog do path
sys.path.append(str(Path.cwd().parent))

# Import naszych modułów
import config
from utils.spaces_handler import SpacesHandler, download_data_file, upload_model

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

# Ustawienia
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("✅ Import bibliotek zakończony pomyślnie")

## 1. Wczytanie danych z Digital Ocean Spaces

In [ ]:
# Inicjalizacja handlera Spaces
spaces = SpacesHandler()

# Pobierz dane z Spaces (najpierw trzeba je tam wrzucić!)
data_files = [
    "halfmarathon_wroclaw_2024.csv",
    "halfmarathon_wroclaw_2023.csv"
]

# Utwórz katalog data jeśli nie istnieje
config.DATA_DIR.mkdir(exist_ok=True)

# Pobierz pliki
for filename in data_files:
    local_path = config.DATA_DIR / filename
    print(f"Pobieranie {filename}...")
    download_data_file(filename, str(local_path))
    
print("✅ Dane pobrane z Digital Ocean Spaces")

In [ ]:
# Wczytaj dane
df_2024 = pd.read_csv(config.DATA_DIR / "halfmarathon_wroclaw_2024.csv", sep=";")
df_2023 = pd.read_csv(config.DATA_DIR / "halfmarathon_wroclaw_2023.csv", sep=";")

# Dodaj kolumnę roku
df_2024['year'] = 2024
df_2023['year'] = 2023

# Połącz dane
df = pd.concat([df_2024, df_2023], ignore_index=True)

print(f"Łączna liczba rekordów: {len(df)}")
print(f"Liczba kolumn: {len(df.columns)}")
print(f"\nPierwsze wiersze:")
df.head()

## 2. Eksploracja i czyszczenie danych

In [ ]:
# Podstawowe statystyki
print("📊 Podstawowe informacje o danych:")
print(df.info())
print("\n" + "="*80)
print("Braki danych:")
print(df.isnull().sum()[df.isnull().sum() > 0])

In [ ]:
# Funkcja do konwersji czasu na sekundy
def time_to_seconds(time_str):
    """Konwertuje czas w formacie HH:MM:SS lub MM:SS na sekundy."""
    if pd.isna(time_str):
        return np.nan
    
    try:
        parts = str(time_str).split(':')
        if len(parts) == 3:
            h, m, s = map(int, parts)
            return h * 3600 + m * 60 + s
        elif len(parts) == 2:
            m, s = map(int, parts)
            return m * 60 + s
        else:
            return np.nan
    except:
        return np.nan

# Konwersja kolumn czasowych
df['time_5km_seconds'] = df['5 km Czas'].apply(time_to_seconds)
df['time_10km_seconds'] = df['10 km Czas'].apply(time_to_seconds)
df['time_15km_seconds'] = df['15 km Czas'].apply(time_to_seconds)
df['time_20km_seconds'] = df['20 km Czas'].apply(time_to_seconds)
df['total_time_seconds'] = df['Czas'].apply(time_to_seconds)

# Oblicz wiek
df['age'] = 2024 - df['Rocznik']

# Enkodowanie płci
le_gender = LabelEncoder()
df['gender_encoded'] = le_gender.fit_transform(df['Płeć'])

print("✅ Konwersja danych zakończona")
print(f"\nMapping płci: {dict(zip(le_gender.classes_, le_gender.transform(le_gender.classes_)))}")

In [ ]:
# Usuń wiersze z brakującymi wartościami w kluczowych kolumnach
df_clean = df.dropna(subset=['time_5km_seconds', 'total_time_seconds', 'age', 'Płeć']).copy()

# Usuń outliers (np. czasy poniżej 1h lub powyżej 4h)
df_clean = df_clean[
    (df_clean['total_time_seconds'] >= 3600) &  # min 1h
    (df_clean['total_time_seconds'] <= 14400) &  # max 4h
    (df_clean['time_5km_seconds'] >= 600) &  # min 10min na 5km
    (df_clean['time_5km_seconds'] <= 2400) &  # max 40min na 5km
    (df_clean['age'] >= 16) &  # min wiek 16
    (df_clean['age'] <= 80)  # max wiek 80
]

print(f"Liczba rekordów po czyszczeniu: {len(df_clean)}")
print(f"Usunięto {len(df) - len(df_clean)} rekordów ({(len(df) - len(df_clean))/len(df)*100:.2f}%)")

In [ ]:
# Wizualizacje
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Rozkład czasu ukończenia
axes[0, 0].hist(df_clean['total_time_seconds']/60, bins=50, edgecolor='black')
axes[0, 0].set_xlabel('Czas ukończenia (minuty)')
axes[0, 0].set_ylabel('Liczba zawodników')
axes[0, 0].set_title('Rozkład czasu ukończenia półmaratonu')

# Czas 5km vs. całkowity czas
axes[0, 1].scatter(df_clean['time_5km_seconds']/60, df_clean['total_time_seconds']/60, alpha=0.3)
axes[0, 1].set_xlabel('Czas na 5km (minuty)')
axes[0, 1].set_ylabel('Całkowity czas (minuty)')
axes[0, 1].set_title('Zależność: Czas 5km vs. Całkowity czas')

# Rozkład wieku
axes[1, 0].hist(df_clean['age'], bins=30, edgecolor='black')
axes[1, 0].set_xlabel('Wiek')
axes[1, 0].set_ylabel('Liczba zawodników')
axes[1, 0].set_title('Rozkład wieku zawodników')

# Czas według płci
df_clean.boxplot(column='total_time_seconds', by='Płeć', ax=axes[1, 1])
axes[1, 1].set_xlabel('Płeć')
axes[1, 1].set_ylabel('Czas (sekundy)')
axes[1, 1].set_title('Czas ukończenia według płci')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 3. Feature Engineering i selekcja cech

In [ ]:
# Dodatkowe cechy
df_clean['pace_5km'] = df_clean['time_5km_seconds'] / 5000  # sekundy na metr
df_clean['age_group'] = pd.cut(df_clean['age'], bins=[0, 25, 35, 45, 55, 100], 
                                labels=['<25', '25-35', '35-45', '45-55', '55+'])

# Analiza korelacji
features_for_corr = ['age', 'gender_encoded', 'time_5km_seconds', 'pace_5km', 'total_time_seconds']
correlation_matrix = df_clean[features_for_corr].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Macierz korelacji cech')
plt.tight_layout()
plt.show()

print("\nKorelacja z czasem ukończenia:")
print(correlation_matrix['total_time_seconds'].sort_values(ascending=False))

In [ ]:
# Wybór finalnych cech
# Na podstawie analizy wybieramy najważniejsze cechy
selected_features = ['age', 'gender_encoded', 'time_5km_seconds']
target = 'total_time_seconds'

X = df_clean[selected_features]
y = df_clean[target]

print(f"\n📊 Wybrane cechy: {selected_features}")
print(f"Zmienna docelowa: {target}")
print(f"\nKształt danych:")
print(f"X: {X.shape}")
print(f"y: {y.shape}")

## 4. Podział danych i trenowanie modelu

In [ ]:
# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Zbiór treningowy: {X_train.shape[0]} rekordów")
print(f"Zbiór testowy: {X_test.shape[0]} rekordów")

In [ ]:
# Porównanie różnych modeli
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
}

results = {}

print("🔬 Trenowanie i porównywanie modeli...\n")

for name, model in models.items():
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, 
                                 scoring='neg_mean_absolute_error', n_jobs=-1)
    
    # Trenowanie na pełnym zbiorze treningowym
    model.fit(X_train, y_train)
    
    # Predykcja na zbiorze testowym
    y_pred = model.predict(X_test)
    
    # Metryki
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'model': model,
        'cv_mae': -cv_scores.mean(),
        'mae': mae,
        'rmse': rmse,
        'r2': r2
    }
    
    print(f"{name}:")
    print(f"  CV MAE: {-cv_scores.mean():.2f} sekund ({-cv_scores.mean()/60:.2f} minut)")
    print(f"  Test MAE: {mae:.2f} sekund ({mae/60:.2f} minut)")
    print(f"  Test RMSE: {rmse:.2f} sekund ({rmse/60:.2f} minut)")
    print(f"  R²: {r2:.4f}")
    print()

In [ ]:
# Wybór najlepszego modelu (na podstawie MAE)
best_model_name = min(results, key=lambda x: results[x]['mae'])
best_model = results[best_model_name]['model']

print(f"🏆 Najlepszy model: {best_model_name}")
print(f"MAE: {results[best_model_name]['mae']:.2f} sekund ({results[best_model_name]['mae']/60:.2f} minut)")
print(f"R²: {results[best_model_name]['r2']:.4f}")

## 5. Walidacja i wizualizacja wyników

In [ ]:
# Predykcja
y_pred_best = best_model.predict(X_test)

# Wizualizacja
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Rzeczywiste vs. Przewidywane
axes[0].scatter(y_test/60, y_pred_best/60, alpha=0.5)
axes[0].plot([y_test.min()/60, y_test.max()/60], 
             [y_test.min()/60, y_test.max()/60], 'r--', lw=2)
axes[0].set_xlabel('Rzeczywisty czas (minuty)')
axes[0].set_ylabel('Przewidywany czas (minuty)')
axes[0].set_title(f'Rzeczywiste vs. Przewidywane ({best_model_name})')
axes[0].grid(True, alpha=0.3)

# Rozkład błędów
errors = (y_test - y_pred_best) / 60  # w minutach
axes[1].hist(errors, bins=50, edgecolor='black')
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Błąd predykcji (minuty)')
axes[1].set_ylabel('Liczba przypadków')
axes[1].set_title(f'Rozkład błędów (MAE: {results[best_model_name]["mae"]/60:.2f} min)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nStatystyki błędów:")
print(f"Średni błąd: {errors.mean():.2f} min")
print(f"Mediana błędu: {errors.median():.2f} min")
print(f"Odchylenie standardowe: {errors.std():.2f} min")

In [ ]:
# Feature importance (jeśli model wspiera)
if hasattr(best_model, 'feature_importances_'):
    importance = pd.DataFrame({
        'feature': selected_features,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.barh(importance['feature'], importance['importance'])
    plt.xlabel('Ważność cechy')
    plt.title(f'Ważność cech w modelu {best_model_name}')
    plt.tight_layout()
    plt.show()
    
    print("\nWażność cech:")
    print(importance)

## 6. Zapisanie modelu

In [ ]:
# Przygotuj metadane modelu
model_metadata = {
    'model_name': best_model_name,
    'model_type': type(best_model).__name__,
    'features': selected_features,
    'target': target,
    'mae': results[best_model_name]['mae'],
    'rmse': results[best_model_name]['rmse'],
    'r2': results[best_model_name]['r2'],
    'training_date': datetime.now().isoformat(),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'label_encoder_classes': le_gender.classes_.tolist(),
}

# Pakiet do zapisania
model_package = {
    'model': best_model,
    'label_encoder': le_gender,
    'metadata': model_metadata
}

# Zapisz lokalnie
config.MODELS_DIR.mkdir(exist_ok=True)
local_model_path = config.MODELS_DIR / f"{config.MODEL_NAME}"
joblib.dump(model_package, local_model_path)

print(f"✅ Model zapisany lokalnie: {local_model_path}")
print(f"\nMetadane modelu:")
for key, value in model_metadata.items():
    print(f"  {key}: {value}")

In [ ]:
# Upload do Digital Ocean Spaces
remote_model_name = f"{config.MODEL_VERSION}_{config.MODEL_NAME}"
success = upload_model(str(local_model_path), remote_model_name)

if success:
    print(f"✅ Model przesłany do Digital Ocean Spaces jako: {remote_model_name}")
    
    # Zapisz także jako "latest"
    upload_model(str(local_model_path), f"latest_{config.MODEL_NAME}")
    print(f"✅ Model zapisany również jako: latest_{config.MODEL_NAME}")
else:
    print("❌ Błąd podczas przesyłania modelu do Spaces")

## 7. Test przykładowych predykcji

In [ ]:
# Funkcja pomocnicza do predykcji
def predict_time(age, gender, time_5km_str):
    """Przewiduje czas ukończenia półmaratonu."""
    # Konwersja czasu 5km na sekundy
    parts = time_5km_str.split(':')
    if len(parts) == 2:
        m, s = map(int, parts)
        time_5km_seconds = m * 60 + s
    else:
        raise ValueError("Czas musi być w formacie MM:SS")
    
    # Enkodowanie płci
    gender_encoded = le_gender.transform([gender])[0]
    
    # Przygotowanie danych
    X_pred = pd.DataFrame({
        'age': [age],
        'gender_encoded': [gender_encoded],
        'time_5km_seconds': [time_5km_seconds]
    })
    
    # Predykcja
    pred_seconds = best_model.predict(X_pred)[0]
    
    # Konwersja na format HH:MM:SS
    hours = int(pred_seconds // 3600)
    minutes = int((pred_seconds % 3600) // 60)
    seconds = int(pred_seconds % 60)
    
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

# Testy
print("🧪 Przykładowe predykcje:\n")

test_cases = [
    {"age": 30, "gender": "M", "time_5km": "22:30", "description": "30-letni mężczyzna, 5km w 22:30"},
    {"age": 25, "gender": "K", "time_5km": "25:00", "description": "25-letnia kobieta, 5km w 25:00"},
    {"age": 45, "gender": "M", "time_5km": "20:00", "description": "45-letni mężczyzna, 5km w 20:00"},
    {"age": 35, "gender": "K", "time_5km": "28:00", "description": "35-letnia kobieta, 5km w 28:00"},
]

for test in test_cases:
    pred_time = predict_time(test["age"], test["gender"], test["time_5km"])
    print(f"{test['description']}")
    print(f"  → Przewidywany czas: {pred_time}\n")

## Podsumowanie

✅ Model został wytrenowany i zapisany
✅ Dane zostały przetworzone i wyczyszczone
✅ Model został przesłany do Digital Ocean Spaces

Model jest gotowy do użycia w aplikacji Streamlit!